In [1]:
from mesa.experimental import JupyterViz

from render import make_render_station_vehicle_count_chart, render_model_with_bg, render_model_orientations
from capacity_vanet import VECModel, get_grid, SEED

In [2]:
page = JupyterViz(
    VECModel,
    model_params={"max_capacity": 25, "load_update_interval": 1},
    measures=[
        render_model_with_bg(get_grid()),
        make_render_station_vehicle_count_chart(tail=100),
        #render_model_orientations,
        "TotalSuccessfulHandoverCount",
        "TotalFailedHandoverCount",
        "AvgQoS",
        "MinQoS",
        "GiniLoad",
    ],
    name="VEC Model",
    space_drawer=False,
    seed=SEED,
)

# This is required to render the visualization in the Jupyter notebook
page

Cannot show ipywidgets in text